In [2]:
import numpy as np
from numba import jit,cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
from matplotlib import pyplot as plt
import math,time
import scipy.ndimage.filters
from threading import Thread,Lock
from ipywidgets import Image, HBox
import ipywidgets as widgets
from ipycanvas import Canvas
import io
x = np.ones([5,5])/np.zeros([5,5])
print(x)



class Cleanup():
    def __init__(self):
        self.threads = []
    def add(self,*threads):
        for thread in threads:
            self.threads.append(thread)
    def reset(self):
        for t in self.threads:
            t.isalive=False
            t.join()
        self.threads=[]
    def hard_reset(self):
        for a in self.threads:
            try:
                a.start()
            except:
                pass
        self.reset()
cleaner = Cleanup()


[[inf inf inf inf inf]
 [inf inf inf inf inf]
 [inf inf inf inf inf]
 [inf inf inf inf inf]
 [inf inf inf inf inf]]


D:\Ananconda\envs\tensorflow2\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':


In [8]:
@cuda.jit
def schmix(grid,bonds,JB,ip,jp,nspins,s,rng_states):
    #let i=this.thread.y*4+ip
    #let j=this.thread.x*4+jp
    i,j=cuda.grid(2)
    if i>=s//4 or j>=s//4:
        return
    mynum=i*s+j
    i=i*4+ip
    j=j*4+jp
    
    #let returnval=0;
    
    #//choose the permutation
    mutate = (math.floor(xoroshiro128p_uniform_float32(rng_states, mynum)* 23))+1
    if mutate>23 or mutate<1:return
    #//evaluate current bonds
    
    a21=grid[(i+1)%s][j%s]
    a31=grid[(i+2)%s][j%s]
    
    a42=grid[(i+3)%s][(j+1)%s]
    a43=grid[(i+3)%s][(j+2)%s]
    
    a12=grid[i%s][(j+1)%s]
    a13=grid[i%s][(j+2)%s]
    
    a24=grid[(i+1)%s][(j+3)%s]
    a34=grid[(i+2)%s][(j+3)%s]
     
    
    a22=grid[(i+1)%s][(j+1)%s]
    a23=grid[(i+1)%s][(j+2)%s]
    a32=grid[(i+2)%s][(j+1)%s]
    a33=grid[(i+2)%s][(j+2)%s]
     
    
    #//left and bottom
    oldbonds = bonds[a21][a22]+bonds[a31][a32]+bonds[a42][a32]+bonds[a43][a33]
    #//right and top
    oldbonds+= bonds[a22][a12]+bonds[a23][a13]+bonds[a23][a24]+bonds[a33][a34]
    #//inner
    oldbonds+= bonds[a22][a23]+bonds[a22][a32]+bonds[a33][a32]+bonds[a33][a23]
    
    #//now we schmix up a22,a23,a32,a33
    if mutate == 0:return#[0, 1, 2, 3]
    elif mutate == 1:c22,c23,c32,c33=a22,a23,a33,a32#[0, 1, 3, 2]
    elif mutate == 2:c22,c23,c32,c33=a22,a32,a23,a33#[0, 2, 1, 3]
    elif mutate == 3:c22,c23,c32,c33=a22,a32,a33,a23#[0, 2, 3, 1]
    elif mutate == 4:c22,c23,c32,c33=a22,a33,a23,a32#[0, 3, 1, 2]
    elif mutate == 5:c22,c23,c32,c33=a22,a33,a32,a23#[0, 3, 2, 1]
    elif mutate == 6:c22,c23,c32,c33=a23,a22,a32,a33#[1, 0, 2, 3]
    elif mutate == 7:c22,c23,c32,c33=a23,a22,a33,a32#[1, 0, 3, 2]
    elif mutate == 8:c22,c23,c32,c33=a23,a32,a22,a33#[1, 2, 0, 3]
    elif mutate == 9:c22,c23,c32,c33=a23,a32,a33,a22#[1, 2, 3, 0]
    elif mutate == 10:c22,c23,c32,c33=a23,a33,a22,a32#[1, 3, 0, 2]
    elif mutate == 11:c22,c23,c32,c33=a23,a33,a32,a22#[1, 3, 2, 0]
    elif mutate == 12:c22,c23,c32,c33=a32,a22,a23,a33#[2, 0, 1, 3]
    elif mutate == 13:c22,c23,c32,c33=a32,a22,a33,a23#[2, 0, 3, 1]
    elif mutate == 14:c22,c23,c32,c33=a32,a23,a22,a33#[2, 1, 0, 3]
    elif mutate == 15:c22,c23,c32,c33=a32,a23,a33,a22#[2, 1, 3, 0]
    elif mutate == 16:c22,c23,c32,c33=a32,a33,a22,a23#[2, 3, 0, 1]
    elif mutate == 17:c22,c23,c32,c33=a32,a33,a23,a22#[2, 3, 1, 0]
    elif mutate == 18:c22,c23,c32,c33=a33,a22,a23,a32#[3, 0, 1, 2]
    elif mutate == 19:c22,c23,c32,c33=a33,a22,a32,a23#[3, 0, 2, 1]
    elif mutate == 20:c22,c23,c32,c33=a33,a23,a22,a32#[3, 1, 0, 2]
    elif mutate == 21:c22,c23,c32,c33=a33,a23,a32,a22#[3, 1, 2, 0]
    elif mutate == 22:c22,c23,c32,c33=a33,a32,a22,a23#[3, 2, 0, 1]
    elif mutate == 23:c22,c23,c32,c33=a33,a32,a23,a22#[3, 2, 1, 0]
    else:return
    
    
    
    newbonds = a21*c22+a31*c32+a42*c32+a43*c33
    #//right and top
    newbonds+= c22*a12+c23*a13+c23*a24+c33*a34
    #//inner
    newbonds+= c22*c23+c22*c32+c33*c32+c33*c23
    
    
    
    #//re-evaluate all the bonds
    newbonds = bonds[a21][c22]+bonds[a31][c32]+bonds[a42][c32]+bonds[a43][c33]
    #//right and top
    newbonds+= bonds[c22][a12]+bonds[c23][a13]+bonds[c23][a24]+bonds[c33][a34]
    #//inner
    newbonds+= bonds[c22][c23]+bonds[c22][c32]+bonds[c33][c32]+bonds[c33][c23]
    
    delta = newbonds-oldbonds
    #//update rule for MCMC
    #//I have very little trust in the GPU Math.random function
    
    #//forcing super unlikely things to not happen   1.000000000001
    n=xoroshiro128p_uniform_float32(rng_states, mynum)
    if(delta < 0 or 1.0-n<math.exp(-JB*delta)):#{
        grid[(i+1)%s][(j+1)%s]=c22
        grid[(i+1)%s][(j+2)%s]=c23
        grid[(i+2)%s][(j+1)%s]=c32
        grid[(i+2)%s][(j+2)%s]=c33
    #returnval=mutate
    #}

    #return returnval;

In [35]:
@cuda.jit
def upscale(inarr,outarr):
    i,j=cuda.grid(2)
    iscale=outarr.shape[0]//inarr.shape[0]
    jscale=outarr.shape[1]//inarr.shape[1]
    outarr[i][j]=inarr[i//iscale][j//jscale]
class Render(Thread):
    def __init__(self, globalmem, canvas,dim=[512,512]):
        self.upscaled=np.zeros(dim)
        self.threads=(16,16)
        self.blocks=(int(np.ceil(dim[0] / 16)),int(np.ceil(dim[1] / 16)))
        self.grid_global_mem = globalmem
        self.canvas = canvas
        self.isalive=True
        super(Render, self).__init__()
    def run(self):
        while self.isalive:
            upscale[self.blocks,self.threads](self.grid_global_mem,self.upscaled)
            gridf=self.upscaled
            pink=(gridf==3)
            blue_channel = (gridf==1)*255+pink*128
            red_channel = (gridf==0)*255+pink*255
            green_channel = (gridf==2)*255+pink*128
            image_data = np.stack((red_channel, green_channel, blue_channel), axis=2)
            self.canvas.put_image_data(image_data, 0, 0)
            time.sleep(0.01)

class Tester(Thread):
    def __init__(self, N):
        self.N=N
        self.grid=np.asarray(np.random.random([N,N])>0.7,dtype=np.int8)
        self.threadsperblock = (16, 16)#should end up a multiple of 32 I think
        blockspergrid_x = int(np.ceil(self.grid.shape[0] / self.threadsperblock[0]))
        blockspergrid_y = int(np.ceil(self.grid.shape[1] / self.threadsperblock[1]))
        self.blockspergrid = (blockspergrid_x, blockspergrid_y)
        self.isalive=True
        self.rng_states = create_xoroshiro128p_states(self.grid.size, seed=1)
        self.grid_global_mem = cuda.to_device(self.grid)
        self.JB=5

        super(Tester, self).__init__()
    def fps(self):
        iold = self.index
        time.sleep(1)
        return self.index-iold
    def run(self):
        grid=self.grid_global_mem
        S1=self.N
        self.index=0
        bds=np.zeros([4,4],dtype=np.float32)
        bds[0][0]=-1
        bds[3][0]=bds[0][3]=1
        bds[2][1]=bds[1][2]=-1
        bds[3][3]=-1
        print(bds)
        bonds=cuda.to_device(bds)

        while self.isalive:
            self.index+=1
            #D=np.random.randint(0,max_num+1)
            threadsperblock = self.threadsperblock
            blockspergrid = (self.blockspergrid[0]//4,self.blockspergrid[1]//4)
            
            ip=np.random.randint(4)
            jp=np.random.randint(4)
            schmix[blockspergrid,threadsperblock](grid,bonds,self.JB,ip,jp,2,S1,self.rng_states)
            #time.sleep(0.3)
            
            
            
cleaner.hard_reset()

model = Tester(256)
model.grid=np.asarray(np.random.random(model.grid.shape)>1.0,dtype=np.int8)#0.878
model.grid+=np.arange(256*256).reshape([256,256])%4
model.grid_global_mem = cuda.to_device(model.grid)
print(model.grid_global_mem.size)
print(np.sum(model.grid_global_mem.copy_to_host()))
model.JB=6
print(model.blockspergrid)
print(model.threadsperblock)
canvas = Canvas(width=512, height=512)
rend = Render(model.grid_global_mem,canvas)
cleaner.add(model,rend)
#model.debug()
#1/0
model.start()
time.sleep(1)
rend.grid_global_mem=model.grid_global_mem
rend.start()

def func(Beta):
    model.JB=Beta
plswork = widgets.Layout(width='50%')
x = widgets.FloatSlider(min=0,max=5,value=1,step=0.001,layout=plswork)
x.style.handle_color = 'lightblue'
widgets.interact(func,Beta=x)


canvas

65536
98304
(16, 16)
(16, 16)
[[-1.  0.  0.  1.]
 [ 0.  0. -1.  0.]
 [ 0. -1.  0.  0.]
 [ 1.  0.  0. -1.]]


interactive(children=(FloatSlider(value=1.0, description='Beta', layout=Layout(width='50%'), max=5.0, step=0.0…

Canvas(height=512, width=512)

In [37]:
cleaner.reset()

In [36]:
model.fps()

3334